In [4]:
import matplotlib.pylab as plt
import numpy as np
import matplotlib.cm as cm
import intake

import numcodecs
try:
    import gribscan
except:
    %pip install gribscan
    import gribscan
import cfgrib
import gc

from getpass import getuser # Libaray to copy things
from pathlib import Path # Object oriented libary to deal with paths
from dask.utils import format_bytes
from dask.diagnostics import ProgressBar

from distributed import Client, progress, wait # Libaray to orchestrate distributed resources
#from dask_jobqueue import SLURMCluster # Setting up distributed memories via slurm
import numpy as np # Pythons standard array library
import xarray as xr # Libary to work with labeled n-dimensional data
import dask # Distributed data libary
import dask.distributed
import multiprocessing
from subprocess import run, PIPE
import sys
import os
import warnings
warnings.filterwarnings(action='ignore')
dask.config.config.get('distributed').get('dashboard').update({'link':'{JUPYTERHUB_SERVICE_PREFIX}/proxy/{port}/status'})
import pandas as pd
import eccodes
try:
    import healpy
except:
    %pip install healpy

import json
import freva

Defaulting to user installation because normal site-packages is not writeable
     l     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/10.0 MB ? eta -:--:--━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/10.0 MB 3.6 MB/s eta 0:00:03━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.7/10.0 MB 10.1 MB/s eta 0:00:01━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/10.0 MB 27.6 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━ 8.7/10.0 MB 63.8 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 10.0/10.0 MB 69.2 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 56.7 MB/s eta 0:00:00
     l     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.9 MB ? eta -:--:--━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 127.6 MB/s eta 0:00:00
     l     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/738.7 kB ? eta -:--:--━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.7/738.7 kB 81.0 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-

In [13]:
#ERA5 dataset can be curated using this link: https://nextgems.dkrz.de/databrowser/
#Will retrieve ERA5 precipitation from 1940-2024
files = freva.databrowser(ensemble="r1i1p1", experiment="era5", institute="ecmwf", model="ifs", product="reanalysis", project="reanalysis", realm="atmos", time_aggregation="mean", time_frequency="1hr", variable="pr")

In [14]:
dataset = xr.open_mfdataset(files, engine='netcdf4',combine='by_coords')


In [15]:
print(dataset)

<xarray.Dataset>
Dimensions:   (time: 743153, lon: 1280, bnds: 2, lat: 640)
Coordinates:
  * time      (time) datetime64[ns] 1940-01-01T07:00:00 ... 2024-10-10T23:00:00
  * lon       (lon) float64 0.0 0.2812 0.5625 0.8438 ... 358.9 359.2 359.4 359.7
  * lat       (lat) float64 -89.78 -89.51 -89.23 -88.95 ... 89.23 89.51 89.78
Dimensions without coordinates: bnds
Data variables:
    lon_bnds  (time, lon, bnds) float64 dask.array<chunksize=(738, 1280, 2), meta=np.ndarray>
    lat_bnds  (time, lat, bnds) float64 dask.array<chunksize=(738, 640, 2), meta=np.ndarray>
    pr        (time, lat, lon) float32 dask.array<chunksize=(738, 640, 1280), meta=np.ndarray>
Attributes:
    CDI:            Climate Data Interface version 1.9.6 (http://mpimet.mpg.d...
    history:        Thu May 16 18:28:51 2024: cdo -s -z zip_9 -mergetime -sel...
    institution:    European Centre for Medium-Range Weather Forecasts
    Conventions:    CF-1.6
    license:        Contains modified Copernicus Atmosphere Monit